# Siamese Convolutional Neural Network

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
# Load the required libraries
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Flatten, Lambda, Dense, Conv2D, Conv2DTranspose
from keras.layers import ZeroPadding2D, Activation, Input, Concatenate, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras import backend as K

import cv2
import os
import numpy as np
import pandas as pd
%tensorflow_version 1.x
import tensorflow as tf
from matplotlib import pyplot

Using TensorFlow backend.


In [0]:
'''
Build the model to generate the features with.
'''

width, height = 224, 224
threshold = 100
emergency = False

# Load a popular model
premodel = VGG16(weights='imagenet', include_top=False, input_shape = (width, height, 3))

# Add a custom layer
flatten = Flatten()(premodel.output)

# Create the final model
model = Model(input = premodel.input, output = flatten)

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [7]:
'''
Generate the features of each image pair and calculate their differences.
'''

# Location of the frame images
folder_path = '/content/drive/My Drive/Masters/Colab Notebooks/Research Code/data/'
folder = 'positive-night/frames-1m/'

counter = 0         # will be counting the iterations and number of frames to use.
evaluate = True

step = 1            # used to calculate the number of frames to use
feature_array = np.array([])

fps = 1                     # frames per seconds
total_frames = 70 * fps     # total frames to read in N seconds

'''
Run the main task to generate features and their distances. Save the distances
in a file and export.
'''
while evaluate:
    # Load and preprocess two images
    img_a_path = folder_path + folder + 'frame%d.jpg' % counter
    img_b_path = folder_path + folder + 'frame%d.jpg' % (counter + step)

    # Load the images
    img_a = image.load_img(img_a_path, target_size=(width, height))
    img_b = image.load_img(img_b_path, target_size=(width, height))

    # Convert images to arrays
    img_a = image.img_to_array(img_a)
    img_b = image.img_to_array(img_b)

    # Insert a new axis to match the model requirements
    img_a = np.expand_dims(img_a, axis=0)
    img_b = np.expand_dims(img_b, axis=0)

    # Preprocess images
    img_a = preprocess_input(img_a)
    img_b = preprocess_input(img_b)

    img_a_feat = model.predict(img_a)
    img_b_feat = model.predict(img_b)

    feat_dist = np.linalg.norm(img_a_feat - img_b_feat)

    feature_array = np.append(feature_array, feat_dist)

    counter = counter + step

    if counter == total_frames - step:
        evaluate = False

    print('Iteration %d' % counter)

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69


In [8]:
feature_array

array([ 66.18109131,  75.54681396,  94.19431305, 102.42622375,
        75.74346161,  82.87364197,  78.03185272,  69.38702393,
        70.13698578,  72.67848969,  70.48339081,  71.69989777,
        65.64519501,  70.43317413,  75.49055481,  72.7075119 ,
        73.57082367,  75.11322784,  80.31594086,  79.13655853,
        90.64601135,  87.11334991,  79.40665436,  71.67411041,
        71.30887604,  74.53318024,  71.37589264,  74.47645569,
        66.16573334,  77.13149261,  79.43213654,  81.08868408,
        77.30462646,  79.87911987,  71.54701996,  71.51528931,
        70.31373596,  69.16754913,  73.54438019,  70.35623932,
        73.55535889,  82.46200562,  85.59383392,  75.27342987,
        82.92714691,  78.64694214,  71.4214325 ,  70.13009644,
        76.92298889,  70.10515594,  73.39225769,  73.92232513,
        73.790802  ,  74.03264618,  72.11442566,  68.85128021,
        74.90722656,  72.16815186,  73.17760468, 109.1835556 ,
        68.6444397 ,  78.01077271,  70.271698  ,  70.00

In [0]:
np.savetxt(folder_path + 'features/pos2neg-night.csv', feature_array, delimiter=',')

In [0]:
feat_array_1 = np.genfromtxt(folder_path + 'features/positive-night.csv', delimiter=',')
feat_array_2 = np.genfromtxt(folder_path + 'features/negative-night.csv', delimiter=',')
feat_array_3 = np.genfromtxt(folder_path + 'features/pos2neg-night.csv', delimiter=',')

In [0]:
# Count the number of noisy elements
print(feature_array[np.where(feature_array >= 185)])

# Remove noise from the data
feature_array = feature_array[feature_array < 220]

# Find an average
feat_avg = np.average(feature_array)

In [12]:
import plotly.graph_objects as go

#seconds = np.arange(0,np.size(feat_array_1)-9)
seconds = np.arange(0,60)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=seconds, y=feature_array[:60], mode='lines', name='Breathing',
                         line = dict(width=2, color='OrangeRed')))

# fig.add_trace(go.Scatter(x=seconds, y=feat_array_2[:60], mode='lines', name='Nonbreathing',
#                          line = dict(width=2, color='SteelBlue')))

# Add a marker and text to the end of the positive line

fig.add_trace(go.Scatter(x=[seconds[-1]],
                        y=[feature_array[:60][-1]],
                        mode='lines+markers+text', 
                        name='test',
                        cliponaxis=False,
                        text=['Breathing'],
                        textfont=dict(size=12),
                        textposition="middle right",
                        marker=dict(size=10, color='OrangeRed')
                        ))


# Add a marker and text to the end of the negative line
# fig.add_trace(go.Scatter(x=[seconds[-1]],
#                         y=[feat_array_2[:60][-1]],
#                         mode='lines+markers+text', 
#                         name='test',
#                         cliponaxis=False,
#                         text=['Nonbreathing'],
#                         textfont=dict(size=12),
#                         textposition="middle right",
#                         marker=dict(size=10, color='SteelBlue')
#                         ))

# Edit the layout
fig.update_layout(xaxis_title='Seconds',
                  yaxis_title='Feature Distance',
                  paper_bgcolor='white',
                  plot_bgcolor='white',
                  showlegend=False,
                  legend_orientation="h"
                  )

fig.update_yaxes(range=[10, 250], nticks=8, gridcolor='Gainsboro', automargin=True)
fig.update_xaxes(range=[1, 60], showgrid=False, nticks=20, ticks="inside", automargin=True)

fig.show(renderer='colab')

In [0]:
# Remove noise from the data
feat_array_1 = feat_array_1[feat_array_1 < 210]
feat_array_2 = feat_array_2[feat_array_2 < 210]
feat_array_1 = feat_array_1[feat_array_1 > 152]
feat_array_2 = feat_array_2[feat_array_2 > 152]
feat_array_3 = feat_array_3[feat_array_3 < 220]
feat_array_3 = feat_array_3[feat_array_3 > 152]

# With average

In [0]:
avg_feat_1 = 60 * [np.average(feat_array_1)]
avg_feat_2 = 60 * [np.average(feat_array_2)]

In [0]:
import plotly.graph_objects as go

#seconds = np.arange(0,np.size(feat_array_1)-9)
seconds = np.arange(0,60)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=seconds, y=feat_array_1[:60], mode='lines', name='Breathing',
                         line = dict(width=2, color='MistyRose'), opacity=0.4))

fig.add_trace(go.Scatter(x=seconds, y=feat_array_2[:60], mode='lines', name='Nonbreathing',
                         line = dict(width=2, color='LightBlue'), opacity=0.4))

fig.add_trace(go.Scatter(x=seconds, y=avg_feat_1, mode='lines', name='Breathing',
                         line = dict(width=2, color='OrangeRed')))

fig.add_trace(go.Scatter(x=seconds, y=avg_feat_2, mode='lines', name='Nonbreathing',
                         line = dict(width=2, color='SteelBlue')))

# Add a marker and text to the end of the negative line
fig.add_trace(go.Scatter(x=[seconds[59]],
                        y=[avg_feat_1[-1]],
                        mode='lines+markers+text', 
                        name='test',
                        cliponaxis=False,
                        text=[np.round(avg_feat_1[-1], 0).astype(int)],
                        textfont=dict(size=11),
                        textposition="middle right",
                        marker=dict(size=11, color='OrangeRed'),
                        showlegend=False
                        ))

fig.add_trace(go.Scatter(x=[seconds[59]],
                        y=[avg_feat_2[-1]],
                        mode='lines+markers+text', 
                        name='test',
                        cliponaxis=False,
                        text=[np.round(avg_feat_2[-1], 0).astype(int)],
                        textfont=dict(size=11),
                        textposition="middle right",
                        marker=dict(size=11, color='SteelBlue'),
                        showlegend=False
                        ))

# Edit the layout
fig.update_layout(xaxis_title='Seconds',
                  yaxis_title='Feature Distance',
                  paper_bgcolor='white',
                  plot_bgcolor='white',
                  showlegend=False,
                  legend_orientation="h"
                  )

fig.update_yaxes(range=[100, 250], nticks=8, gridcolor='Gainsboro', automargin=True)
fig.update_xaxes(range=[1, 60], showgrid=False, nticks=20, ticks="inside", automargin=True)

fig.show(renderer='colab')